In [1]:
! pip install pytorch_pretrained_bert
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 10.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.8 MB/s eta 0:00:00


In [2]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import numpy as np
import pandas as pd
import json

def get_sentiment(txt):
    labels = { 0:'neutral', 1:'positive', 2:'negative' }
    inputs = roberta_tokenizer(txt, return_tensors="pt", padding=True, truncation='longest_first')
    outputs = roberta(**inputs)[0]
    print(outputs)
    val = labels[np.argmax(outputs.detach().numpy())]
    return val

def label_articles(path):
    count = 0
    result = []
    with open(path, "r") as f:
        data = json.load(f)
    for item in data:
        if count == CONST_NUM_ARTICLES:
          break
        count += 1
        txt = item['text']
        val = get_sentiment(txt)
        item['sentiment'] = val
        result.append(item)
    return result

CONST_NUM_ARTICLES = 10 # Change this depending on how many articles you want to label

from transformers import AutoTokenizer, AutoModelForSequenceClassification

roberta_tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

roberta = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

res = label_articles('/content/drive/MyDrive/CMPE CISC 351 Project/Data/Trading_benchmark/evaluate_news.json')

json_object = json.dumps(res, indent=2)

with open("labelled_benchmark.json", "w") as outfile:
  outfile.write(json_object)

tensor([[-3.2551, -1.5387,  5.3755]], grad_fn=<AddmmBackward0>)
tensor([[-4.1439,  4.3596,  0.5550]], grad_fn=<AddmmBackward0>)
tensor([[-3.6394,  5.6204, -0.8892]], grad_fn=<AddmmBackward0>)
tensor([[-2.6122, -3.1213,  5.8761]], grad_fn=<AddmmBackward0>)
tensor([[ 4.5979, -3.5398, -1.5523]], grad_fn=<AddmmBackward0>)
tensor([[-2.7796, -2.5018,  5.6875]], grad_fn=<AddmmBackward0>)
tensor([[-4.0923,  3.5667,  1.4449]], grad_fn=<AddmmBackward0>)
tensor([[-3.9946,  4.7437,  0.1587]], grad_fn=<AddmmBackward0>)
tensor([[-2.8582, -2.3665,  5.6254]], grad_fn=<AddmmBackward0>)
tensor([[-3.7523,  6.2020, -1.5849]], grad_fn=<AddmmBackward0>)


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import json

def get_sentiment(txt):
    inputs = prosus_tokenizer(txt, return_tensors="pt", padding=True, truncation=True)
    outputs = prosus_model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    positive_prob = predictions[:, 0].tolist()[0]
    negative_prob = predictions[:, 1].tolist()[0]
    neutral_prob = predictions[:, 2].tolist()[0]
    pred = max(positive_prob, negative_prob, neutral_prob)
    if (pred == positive_prob) & (pred > CONST_THRESHOLD):
      return 'positive'
    elif (pred == negative_prob) & (pred > CONST_THRESHOLD):
      return 'negative'
    return 'neutral'  # Default case

def label_articles(path):
    count = 0
    result = []
    with open(path, "r") as f:
        data = json.load(f)
    for item in data:
        if count == CONST_NUM_ARTICLES:
          break
        count += 1
        txt = item['text']
        val = get_sentiment(txt)
        item['sentiment'] = val
        result.append(item)
    return result

CONST_NUM_ARTICLES = 1  # Change this depending on how many articles you want to label
CONST_THRESHOLD = 0.5   # Change this according to your prediction tolerance (probability cutoff)

prosus_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

prosus_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

res = label_articles('/content/drive/MyDrive/CMPE CISC 351 Project/Data/Trading_benchmark/evaluate_news.json')

json_object = json.dumps(res, indent=2)

with open("labelled_benchmark.json", "w") as outfile:
  outfile.write(json_object)

In [ ]:
from transformers import pipeline
import pandas as pd
import numpy as np
import torch
import json

def get_sentiment(txt):
    pred = roberta_model(txt[0:1000], truncation = True)[0]
    if pred['score'] > CONST_THRESHOLD:
      return pred['label']
    return 'neutral'

def label_articles(path):
    count = 0
    result = []
    with open(path, "r") as f:
        data = json.load(f)
    for item in data:
        if count == CONST_NUM_ARTICLES:
          break
        count += 1
        txt = item['text']
        val = get_sentiment(txt)
        item['sentiment'] = val
        result.append(item)
    return result

CONST_NUM_ARTICLES = 1  # Change this depending on how many articles you want to label
CONST_THRESHOLD = 0.5   # Change this according to your prediction tolerance (probability cutoff)

roberta_model = pipeline("sentiment-analysis", model="soleimanian/financial-roberta-large-sentiment")

res = label_articles('/content/drive/MyDrive/CMPE CISC 351 Project/Data/Trading_benchmark/evaluate_news.json')

json_object = json.dumps(res, indent=2)

with open("labelled_benchmark.json", "w") as outfile:
  outfile.write(json_object)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
def predict():
  sent_val = list()
  i = 0
  true_count = 0
  for count, row in df_articles.iterrows():
      txt = row['text']
      txt = txt[0:1500]
      true_label = row['label']
      if true_label == 1:
        true_label = 1
      elif true_label == 0.5:
        true_label = 0
      elif true_label == 0:
        true_label = 2
      true_label = labels[true_label]
      inputs = tokenizer(txt, return_tensors="pt", padding=True)
      outputs = finbert(**inputs)[0]

      if true_label == val:
        true_count += 1
      i += 1

      val = labels[np.argmax(outputs.detach().numpy())]
      print(true_label, '----', val)
      print(row['text'])
      print('#######################################################')    
      sent_val.append(val)

  print("count of correctly classified: " + str(true_count))
  print("total count: " + str(i))

In [ ]:
# FinBERT PERFORMANCE ON EDT

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import numpy as np
import torch
import json

def get_articles(path):
  df = pd.read_csv(path)
  articles = []
  sentiments = []
  for i, row in df.iterrows():
    articles.append(row['Article Text'])
    if row['Sentiment Label'] == 'Positive':
      sentiments.append("positive")
    elif row['Sentiment Label'] == 'Neutral':
      sentiments.append("neutral")
    else:
      sentiments.append("negative")
  df_dict = {
      'text' : articles,
      'label' : sentiments
  }
  df = pd.DataFrame.from_dict(df_dict)
  return df

def get_finbert_sentiment(txt, prosus_tokenizer, prosus_model):
    inputs = prosus_tokenizer(txt, return_tensors="pt", padding=True, truncation=True)
    outputs = prosus_model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    positive_prob = predictions[:, 0].tolist()[0]
    negative_prob = predictions[:, 1].tolist()[0]
    neutral_prob = predictions[:, 2].tolist()[0]
    pred = max(positive_prob, negative_prob, neutral_prob)
    if (pred == positive_prob) & (pred > CONST_THRESHOLD):
      return 'positive'
    elif (pred == negative_prob) & (pred > CONST_THRESHOLD):
      return 'negative'
    return 'neutral'  # Default case

def predict(prosus_tokenizer, prosus_model):
  true_count = 0
  total_count = 75
  neg_count = 0
  total_neg = 10
  pos_count = 0
  total_pos = 54
  for count, row in df_articles.iterrows():
      txt = row['text']
      txt = txt[0:1500]
      true_label = row['label']
      predicted_label = get_finbert_sentiment(txt, prosus_tokenizer, prosus_model)
      if (true_label.lower() == predicted_label) & (predicted_label == "positive"):
        pos_count += 1
      if (true_label.lower() == predicted_label) & (predicted_label == "negative"):
        neg_count += 1
      if true_label.lower() == predicted_label:
        true_count += 1
      #print(true_label, '----', predicted_label)
      #print(row['text'])
      #print('#######################################################')    
  
  print("count of correctly classified: " + str(true_count))
  print("total count: " + str(total_count))
  print("Accuracy: " + str(true_count/total_count))
  print("True Positive Rate: " + str(pos_count/total_pos))
  print("True Negative Rate: " + str(neg_count/total_neg))

df_articles = get_articles('/content/drive/MyDrive/CMPE CISC 351 Project/Sentiment_Labelling.csv')

CONST_THRESHOLD = 0.5   # Change this according to your prediction tolerance (probability cutoff)

prosus_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
prosus_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

predict(prosus_tokenizer, prosus_model)

count of correctly classified: 33
total count: 75
Accuracy: 0.44
True Positive Rate: 0.42592592592592593
True Negative Rate: 0.1


In [ ]:
# ROBERTA PERFORMANCE ON EDT

from transformers import pipeline
import pandas as pd
import numpy as np
import torch
import json

def get_articles(path):
  df = pd.read_csv(path)
  articles = []
  sentiments = []
  for i, row in df.iterrows():
    articles.append(row['Article Text'])
    if row['Sentiment Label'] == 'Positive':
      sentiments.append("Positive")
    elif row['Sentiment Label'] == 'Neutral':
      sentiments.append("Neutral")
    else:
      sentiments.append("Negative")
  df_dict = {
      'text' : articles,
      'label' : sentiments
  }
  df = pd.DataFrame.from_dict(df_dict)
  return df

def get_roberta_sentiment(txt):
    pred = roberta_model(txt[0:1000], truncation = True)[0]
    if pred['score'] > CONST_THRESHOLD:
      return pred['label']
    return 'neutral'

def predict(roberta_model):
  sent_val = list()
  true_count = 0
  total_count = 75
  neg_count = 0
  total_neg = 10
  pos_count = 0
  total_pos = 54
  for count, row in df_articles.iterrows():
      txt = row['text']
      txt = txt[0:1500]
      true_label = row['label']
      predicted_label = get_roberta_sentiment(txt)
      if (true_label.lower() == predicted_label) & (predicted_label == "positive"):
        pos_count += 1
      if (true_label.lower() == predicted_label) & (predicted_label == "negative"):
        neg_count += 1
      if true_label.lower() == predicted_label:
        true_count += 1
      #print(true_label, '----', predicted_label)
      #print(row['text'])
      #print('#######################################################')    
  
  print("count of correctly classified: " + str(true_count))
  print("total count: " + str(total_count))
  print("Accuracy: " + str(true_count/total_count))
  print("True Positive Rate: " + str(pos_count/total_pos))
  print("True Negative Rate: " + str(neg_count/total_neg))

df_articles = get_articles('/content/drive/MyDrive/CMPE CISC 351 Project/Sentiment_Labelling.csv')

CONST_THRESHOLD = 0.5   # Change this according to your prediction tolerance (probability cutoff)

roberta_model = pipeline("sentiment-analysis", model="soleimanian/financial-roberta-large-sentiment")

predict(roberta_model)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


count of correctly classified: 54
total count: 75
Accuracy: 0.72
True Positive Rate: 0.9259259259259259
True Negative Rate: 0.2
